In [ ]:
from pydoc import describe
from xml.etree.ElementInclude import include
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np 
import seaborn as sns
import missingno as msno
from datetime import datetime, date
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier 
from sklearn.impute import KNNImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score, roc_auc_score
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from sklearn.inspection import permutation_importance

pd.set_option('display.max_columns', 10000)




train1= pd.read_csv(r"C:\Users\terli\Programs\Python\Projects\Advanced Business Analytics\train_month_1.csv")
train2= pd.read_csv(r"C:\Users\terli\Programs\Python\Projects\Advanced Business Analytics\train_month_2.csv")
train3= pd.read_csv(r"C:\Users\terli\Programs\Python\Projects\Advanced Business Analytics\train_month_3_with_target.csv")

df= pd.concat([train1.add_suffix('_1'),train2.add_suffix('_2'),train3],axis=1)

TT='2018-01'
TT=pd.to_datetime(TT,yearfirst=True,format='%Y-%m')

imputer=KNNImputer(n_neighbors=5,weights='uniform')

df=df.drop(df[['client_id','client_id_1','client_id_2']],axis=1)

In [ ]:
test_size=0.25
holdout_size=0.05


In [ ]:
dataset=df


In [ ]:
dataset.customer_children=dataset.customer_children.replace(['no','mature','adolescent','young','preschool','grownup','onebaby','yes',],[0,1,2,3,4,5,6,7])
dataset.customer_children_2=dataset.customer_children_2.replace(['no','mature','adolescent','young','preschool','grownup','onebaby','yes',],[0,1,2,3,4,5,6,7])
dataset.customer_children_1=dataset.customer_children_1.replace(['no','mature','adolescent','young','preschool','grownup','onebaby','yes',],[0,1,2,3,4,5,6,7])
#Relationship, string to integer
dataset.customer_relationship=dataset.customer_relationship.replace(['couple','single'],[0,1])
dataset.customer_relationship_2=dataset.customer_relationship_2.replace(['couple','single'],[0,1])
dataset.customer_relationship_1=dataset.customer_relationship_1.replace(['couple','single'],[0,1])

In [ ]:
dataset['Age']=(((TT)-(pd.to_datetime(dataset.customer_birth_date, yearfirst=True,format='%Y-%m')))/365.25).dt.days
#Client all since
dataset['Years_Since_all']=(((TT)-(pd.to_datetime(dataset.customer_since_bank, yearfirst=True,format='%Y-%m')))/365.25).dt.days
#Client Bank since
dataset['Years_Since_bank']=(((TT)-(pd.to_datetime(dataset.customer_since_all, yearfirst=True,format='%Y-%m')))/365.25).dt.days

In [ ]:
dataset=dataset.drop(dataset[dataset['Age']>100].index)

In [ ]:
print(dataset.customer_children.isna().sum(),dataset.customer_children_1.isna().sum(),dataset.customer_children_2.isna().sum())

In [ ]:
dataset.customer_children_2=dataset.customer_children_2.fillna(dataset['customer_children_1'])
dataset.customer_children=dataset.customer_children.fillna(dataset['customer_children_2'])

In [ ]:
dataset.customer_relationship_2=dataset.customer_relationship_2.fillna(dataset['customer_relationship_1'])
dataset.customer_relationship=dataset.customer_relationship.fillna(dataset['customer_relationship_2'])

In [ ]:
dataset=dataset.drop(dataset[dataset['Age']<=dataset.Years_Since_all.mean()].index)
dataset=dataset.drop(dataset[dataset['Age']<=dataset.Years_Since_bank.mean()].index)

In [ ]:
dataset['Years_Since_all']=dataset['Years_Since_all'].fillna(dataset.Years_Since_all.mean())
dataset['Years_Since_bank']=dataset['Years_Since_bank'].fillna(dataset.Years_Since_all.mean())



In [ ]:
dataset['customer_occupation_code']=dataset.customer_occupation_code.where(~dataset.customer_occupation_code.isnull(),(dataset.customer_occupation_code.mode()[0]))
#K-nearest neighbours for missing children and relationship 
dataset[['customer_children','customer_relationship']]=imputer.fit_transform(dataset[['customer_children','customer_relationship']])
#drop education code
dataset=dataset.drop(dataset.filter(like='education'),axis=1)

In [ ]:
#Change in occupation status 
dataset['change_occupation']=dataset.iloc[:,dataset.columns.get_loc('customer_occupation_code')].ne(dataset['customer_occupation_code_1'],axis=0)
print('Change occupation',dataset.shape)
#Change in chilldren status
dataset['change_children']=dataset.iloc[:,dataset.columns.get_loc('customer_children')].ne(dataset['customer_children'],axis=0)
print('Change children',dataset.shape)
#Customer became self employed 
dataset['change_self-employed']=dataset.iloc[:,dataset.columns.get_loc('customer_self_employed')].ne(dataset['customer_self_employed_1'],axis=0)
print('Changed self emmployed',dataset.shape)
#Customer moved out 
dataset['change_Postal-code']=dataset.iloc[:,dataset.columns.get_loc('customer_postal_code')].ne(dataset['customer_postal_code_1'],axis=0)
print('Changed postal code',dataset.shape)

In [ ]:
AccountTT=dataset[['has_homebanking','has_insurance_21','has_insurance_23','has_life_insurance_fixed_cap','has_life_insurance_decreasing_cap','has_fire_car_other_insurance','has_personal_loan','has_mortgage_loan','has_current_account','has_pension_saving','has_savings_account','has_savings_account_starter','has_current_account_starter']]
AccountsTT_2=dataset[['has_homebanking_2','has_insurance_21_2','has_insurance_23_2','has_life_insurance_fixed_cap_2','has_life_insurance_decreasing_cap_2','has_fire_car_other_insurance_2','has_personal_loan_2','has_mortgage_loan_2','has_current_account_2','has_pension_saving_2','has_savings_account_2','has_savings_account_starter_2','has_current_account_starter_2']]
AccountsTT_1=dataset[['has_homebanking_1','has_insurance_21_1','has_insurance_23_1','has_life_insurance_fixed_cap_1','has_life_insurance_decreasing_cap_1','has_fire_car_other_insurance_1','has_personal_loan_1','has_mortgage_loan_1','has_current_account_1','has_pension_saving_1','has_savings_account_1','has_savings_account_starter_1','has_current_account_starter_1']]
dataset['Change_Accounts']=AccountTT.sum(axis=1)-AccountsTT_1.sum(axis=1)
print('Changed naccounts',dataset.shape)

In [ ]:
dataset['Activity']=(dataset[['homebanking_active','homebanking_active_2','homebanking_active_1']]).sum(axis=1)
dataset['Inactivity']=dataset['Activity'].apply(lambda x: 1 if x==0 else 1)


In [ ]:
dataset['change_insurance_21']=dataset['bal_insurance_21']-dataset['bal_insurance_21_2']
dataset['change_insurance_23']=dataset['bal_insurance_23']-dataset['bal_insurance_23_2']
dataset['change_mortgage_loan']=dataset['bal_mortgage_loan']-dataset['bal_mortgage_loan_2']
dataset['change_personal_loan']=dataset['bal_personal_loan']-dataset['bal_personal_loan_2']
dataset['change_current_account']=dataset['bal_current_account']-dataset['bal_current_account_2']
dataset['change_pension_saving']=dataset['bal_pension_saving']-dataset['bal_pension_saving_2']
dataset['change_savings_account']=dataset['bal_savings_account']-dataset['bal_savings_account_2']
dataset['change_current_account_starter']=dataset['bal_current_account_starter']-dataset['bal_current_account_starter_2']
dataset['change_savings_account_starter']=dataset['bal_savings_account_starter']-dataset['bal_savings_account_starter_2']

dataset['change_insurance_21_2']=dataset['bal_insurance_21_2']-dataset['bal_insurance_21_1']
dataset['change_insurance_23_2']=dataset['bal_insurance_23_2']-dataset['bal_insurance_23_1']
dataset['change_mortgage_loan_2']=dataset['bal_mortgage_loan_2']-dataset['bal_mortgage_loan_1']
dataset['change_personal_loan_2']=dataset['bal_personal_loan_2']-dataset['bal_personal_loan_1']
dataset['change_current_account_2']=dataset['bal_current_account_2']-dataset['bal_current_account_1']
dataset['change_pension_saving_2']=dataset['bal_pension_saving_2']-dataset['bal_pension_saving_1']
dataset['change_savings_account_2']=dataset['bal_savings_account_2']-dataset['bal_savings_account_1']
dataset['change_current_account_starter_2']=dataset['bal_current_account_starter_2']-dataset['bal_current_account_starter_1']
dataset['change_savings_account_starter_2']=dataset['bal_savings_account_starter_2']-dataset['bal_savings_account_starter_1']


In [ ]:
target=dataset['target']
dataset= dataset.drop(dataset[['target','customer_postal_code','customer_birth_date','customer_since_all','customer_since_bank', 'customer_since_all_1','customer_since_all_2','customer_since_bank_1','customer_since_bank_2','customer_gender_1','customer_gender_2','customer_birth_date_1','customer_birth_date_2','customer_postal_code_1','customer_postal_code_2','customer_self_employed_1','customer_self_employed_2','customer_occupation_code_1','customer_occupation_code_2','customer_children_1','customer_children_2','customer_relationship_1','customer_relationship_2','has_homebanking_2','has_insurance_21_2','has_insurance_23_2','has_life_insurance_fixed_cap_2','has_life_insurance_decreasing_cap_2','has_fire_car_other_insurance_2','has_personal_loan_2','has_mortgage_loan_2','has_current_account_2','has_pension_saving_2','has_savings_account_2','has_savings_account_starter_2','has_current_account_starter_2','has_homebanking_1','has_insurance_21_1','has_insurance_23_1','has_life_insurance_fixed_cap_1','has_life_insurance_decreasing_cap_1','has_fire_car_other_insurance_1','has_personal_loan_1','has_mortgage_loan_1','has_current_account_1','has_pension_saving_1','has_savings_account_1','has_savings_account_starter_1','has_current_account_starter_1','bal_insurance_21_2','bal_insurance_23_2','bal_mortgage_loan_2','bal_personal_loan_2','bal_current_account_2','bal_pension_saving_2','bal_savings_account_2','bal_current_account_starter_2','bal_savings_account_starter_2','bal_insurance_21_1','bal_insurance_23_1','bal_mortgage_loan_1','bal_personal_loan_1','bal_current_account_1','bal_pension_saving_1','bal_savings_account_1','bal_current_account_starter_1','bal_savings_account_starter_1','cap_life_insurance_fixed_cap_1','cap_life_insurance_fixed_cap_2','cap_life_insurance_decreasing_cap_1','cap_life_insurance_decreasing_cap_2','prem_fire_car_other_insurance_1']],axis=1)


In [ ]:
X_train,X_temp,y_train,y_temp=train_test_split(dataset,target,test_size=test_size+holdout_size)
X_test,X_holdout,y_test,y_holdout=train_test_split(X_temp,y_temp,test_size=holdout_size/(test_size+holdout_size))


In [ ]:

print('length of the training set is', len(X_train))
print('length of the test set is',len(X_test))
print('length of the holdout set is',len(X_holdout))


In [ ]:

samplingstrat=0.5
rus= RandomUnderSampler(sampling_strategy=samplingstrat,random_state=0)
X_train_under, y_train_under=rus.fit_resample(X_train,y_train)

print(X_train_under.shape)
print(y_train_under.shape,y_train_under.sum())


In [ ]:
Rdfclf= RandomForestClassifier(n_estimators=200,random_state=20)
Rdfclf.fit(X_train_under,y_train_under)


y_pred_under=Rdfclf.predict(X_test)

print(confusion_matrix(y_test,y_pred_under))
print(accuracy_score(y_test,y_pred_under))
print(classification_report(y_test,y_pred_under))

In [ ]:
roc_auc_score(y_test,y_pred_under)

In [ ]:
#result = permutation_importance(Rdfclf, X_test, y_test, n_repeats=10, random_state=42, n_jobs=2)


In [ ]:
#feature_names = [f"feature {i}" for i in range(X_test.shape[0])]

In [ ]:

#forest_importances = pd.Series(result.importances_mean, index=feature_names)

In [ ]:
#fig, ax = plt.subplots()
#forest_importances.plot.bar(yerr=result.importances_std, ax=ax)
#ax.set_title("Feature importances using permutation on full model")
#ax.set_ylabel("Mean accuracy decrease")
#fig.tight_layout()
#plt.show()